In [3]:
# 필요 패키지 설치
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install pandas
!pip install numpy==1.23.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 8.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp311-cp311-linux_x86_64.whl size=679485 sha256=782787d16806f217f5ef324fc09cf2d91e1c7d42dca906606e037919bad5f72c
  Stored in directory: /root/.cache/pip/wheels/29/64/e3/047328bfc4a551696adc4a82dba3c980a1dad62198b8a3bb83
Successfully built gluonnlp
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [1]:
#kobert
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-32b7b2ma/kobert-tokenizer_93398854c2ec44f2b2187d08626bd5fe
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-32b7b2ma/kobert-tokenizer_93398854c2ec44f2b2187d08626bd5fe
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [3]:
#  Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
# GPU 설정
device = torch.device("cuda:0")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [7]:
chat_data = pd.read_excel('/content/drive/MyDrive/korean_chat_dataset.xlsx')

In [8]:
# 2~6번째 열 삭제(쓸모없는 열)
chat_data = chat_data.drop(chat_data.columns[[2,3,4,5,6]], axis=1)
chat_data.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포


In [9]:
# Nan인 Item ""로 채우기
chat_data = chat_data.fillna("")

In [10]:
# 공백 삭제
chat_data["Emotion"] = chat_data["Emotion"].apply(lambda x:x.strip())

In [11]:
chat_data["Emotion"].value_counts()

,count
Emotion,
행복,6037
놀람,5898
분노,5665
공포,5468
혐오,5429
슬픔,5267
중립,4830


In [12]:
chat_data.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포


In [13]:
# model 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [14]:
# 7개의 감정 class -> 숫자
chat_data.loc[(chat_data['Emotion']=="공포"), 'Emotion']=0  # 공포 => 0
chat_data.loc[(chat_data['Emotion']=="놀람"), 'Emotion']=1  # 놀람 => 1
chat_data.loc[(chat_data['Emotion']=="분노"), 'Emotion']=2  # 분노 => 2
chat_data.loc[(chat_data['Emotion']=="슬픔"), 'Emotion']=3  # 슬픔 => 3
chat_data.loc[(chat_data['Emotion']=="중립"), 'Emotion']=4  # 중립 => 4
chat_data.loc[(chat_data['Emotion']=="행복"), 'Emotion']=5  # 행복 => 5
chat_data.loc[(chat_data['Emotion']=="혐오"), 'Emotion']=6  # 혐오 => 6

data_list = []
for q, label in zip(chat_data['Sentence'], chat_data['Emotion']):
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

print(data)
print(data_list[:10])

['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']
[['언니 동생으로 부르는게 맞는 일인가요..??', '0'], ['그냥 내 느낌일뿐겠지?', '0'], ['아직너무초기라서 그런거죠?', '0'], ['유치원버스 사고 낫다던데', '0'], ['근데 원래이런거맞나요', '0'], [' 남자친구가 떠날까봐요', '0'], ['이거 했는데 허리가 아플수도 있나요? ;;', '0'], ['내가불안해서꾸는걸까..', '0'], [' 일주일도 안 남았당...ㅠㅠ', '0'], ['약은 최대한 안먹으려고 하는데좋은 음시있나요?0', '0']]


In [15]:
# Train & Test 데이터로 나누기
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=32)

In [16]:
# 데이터셋 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [17]:
# BERTSentenceTransform 클래스 정의
# BERT 모델에 입력으로 넣기 위한 전처리 과정
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

    self.sentences = [transform([i[sent_idx]]) for i in dataset] # 문장 변환
    self.labels = [np.int32(i[label_idx]) for i in dataset] # label 변환

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):  # 전체 데이터셋의 길이 변환
    return (len(self.labels))

In [18]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-6

In [19]:
# Data tokenization, int encoding, padding

# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
tok = tokenizer.tokenize  # 토크나이저 가져옴

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [20]:
# torch 형식의 dataset을 만들어 입력 데이터셋 전처리 마무리
# 테스트를 수행하기 위한 데이터로 로드
train_dataloader = DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## KOBERT 모델 구현

In [36]:
class BERTClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size=768,
               num_classes=7, # 감정 클래스 수로 조정
               dr_rate=0.1,
               params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifer = nn.Linear(hidden_size, num_classes)  # 선형 레이어를 통해 BERT의 출력을 감정 클래스로 매핑
    if dr_rate: # 드롭아웃 레이어를 선택적으로 추가
      self.dropout = nn.Dropout(p=dr_rate)
  # 어텐션 마스크 생성 : 패딩된 부분 가리기
  # 불필요하게 패딩 토큰에 대해서 어텐션하지 않기
  # 1 : 실제 단어이므로 마스킹 하지 않음
  # 0 : 패딩 토큰이므로 마스킹한다
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):  # 예측값 생성
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    # pooler: BERT 모델의 출력 중 하나로, 전체 시퀀스를 요약하는 역할
    _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device), return_dict=False)
    if self.dr_rate:
      out = self.dropout(pooler)
    return self.classifer(out)


In [37]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer와 schedule 설정 (Linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimzier_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimzier_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X, Y):
  max_vals, max_indices = torch.max(X, 1)
  train_acc = (max_indices==Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc

train_dataloader

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [1]:
for e in range(num_epochs):
  train_acc = 0.0
  test_acc = 0.0
  model.train() # 학습 모드로 설정
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):  # 학습 데이터셋을 순회하면서 각 배치에 대해 반복
    optimizer.zero_grad()
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids) # 모델의 데이터를 전달하고 예측 얻기
    loss = loss_fn(out, label)  # 손실 계산, 역전파 수행
    loss.backward() # gradient explore 방지
    nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()  # 모델의 파라미터 업데이트
    scheduler.step()  # Update learning rate schedule
    train_acc += calc_accuracy(out, label)  # 정확도 누적
    if batch_id % log_interval == 0:
      print(f"epoch {e+1} batch id {batch_id+1} loss {loss.data.cpu().numpy()} train acc {train_acc/(batch_id+1)}")
  print(f"epoch {e+1} train acc {batch_id+1}")

  model.eval()  # 평가 모드로 설정
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)): # 테스트 데이셋을 순회하면서 각 배치에 대해 반복
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device) # 필요한 데이터를 GPU로 이동
    out = model(token_ids, valid_length, segment_ids) # 모델에 데이터를 전달하고 예측 얻음
    test_acc += calc_accuracy(out, label) # 정확도 누적
  print(f"epoch {e+1} test acc {test_acc/(batch_id+1)}")

NameError: name 'num_epochs' is not defined

## Predict 함수 정의

In [ ]:
def predict(predict_sentence):  # input = 감정분류 하고자 하는 sentence
  data = [predict_sentence, '0']  # input 준비 및 전처리
  dataset_another = [data]  # predict sentence를 튜플로 변환

  another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # BERT 모델에 입력으로 사용할 데이터셋 생성 : 토큰화한 문장
  test_dataloader = DataLoader(another_test, batch_size=batch_size, num_workers=5)  # torch 형식 변환 : mini batch

  model.eval()  # 모델 평가 모드

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length = valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids) # 모델에 미니배치를 입력으로 주어 예측값을 얻는다
    test_eval = []
    for i in out: # out = model(token_ids, valid_length, segment_ids)
      logits = i
      logits = logits.detach().cpu().numpy()

      # 감정 레이블 및 퍼센트 계산
      emotion_labels = ["공포", "놀람", "분노", "슬픔", "중립", "행복", "혐오"]
      probabilities = [np.exp(logit) / np.sum(np.exp(logits)) * 100 for logit in logits]

      for label, percent in zip(emotion_labels, probabilities):
        print(f"{label} : {percent:.2f}%")

      max_index = np.argmax(logits)
      max_emotion_label = emotion_labels[max_index]
      print(f">> 입력하신 내용에서 {max_emotion_label} 느껴집니다.")

In [ ]:
# 질문에 0 입력시 종료
end = 1
while end == 1:
  sentence = input("하고 싶은 말을 입력해주세여 : ")
  if sentence == "0":
    break
  predict(sentence)
  print("\n")

In [2]:
!pip install onnx

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 362, in run
    resolver = self.make_resolver(
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 177, in make_resolver
    return pip._internal.resolution.resolvelib.resolver.Resolver(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 58, in __init__
    self.factory = Factory(
                   ^^^^^^^^
  File "/usr/local/lib/py

In [ ]:
import torch.onnx

# 더미 입력을 GPU로 이동
dummy_input = (
    torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]).to(device), # token_ids
    torch.LongTensor([10]).to(device), # valid_length
    torch.LongTensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]).to(device) # segment_ids
)

# 모델을 ONNX 형식으로 내보낸다
onnx_path = 'emotion_model_include_neutral.onnx'
torch.onnx.export(model, dummy_input, onnx_path, verbose=True, input_names=['input'], output_names=['output'])

from google.colab import files
files.download('emotion_model_include_neutral.onnx')